# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)


## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Barbecue (BBQ)** in **Austin, Texas**, United States of America.

Since there are lots of restaurants in Austin, TX we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no BBQ restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to BBQ restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **MapQuest API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Austin center will be obtained using **MapQuest API geocoding** of well known Austin location (Texas State Capital)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Austin, Texas city center.

Let's first find the latitude & longitude of Austin, Texas city center, using specific, well known address and MapQuest geocoding API.

In [2]:
import pandas as pd
import requests
import numpy as np
!pip install shapely
!pip install pyproj

print("Libraries Loaded")

Libraries Loaded


In [3]:
# @hidden_cell
# MapQuest credentials
mapquest_api_key = 'ipXsVgVWfeo23U7ddliS7pZecfX9qNGF'
print("MapQuest Credientials Loaded")

MapQuest Credientials Loaded


In [4]:
def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'http://open.mapquestapi.com/geocoding/v1/address?key={}&location={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
           print('MapQuest API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['locations'][0]['latLng'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Texas State Capitol, Austin, TX 78701' # Address of Texas State Capitol
austin_center = get_coordinates(mapquest_api_key, address)
print('Coordinate of {}: {}'.format(address, austin_center))

Coordinate of Texas State Capitol, Austin, TX 78701: [30.274687, -97.740405]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from the Texas State Capitol. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [5]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj
from pyproj import Transformer

import math

def lonlat_to_xy(lon, lat):
    lonlat_to_xy_transformer = Transformer.from_proj({"proj":'latlong',"datum":'WGS84'}, {"proj":"utm", "zone":33, "datum":'WGS84'})
    xy = lonlat_to_xy_transformer.transform(lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    xy_to_lonlat_transformer = Transformer.from_proj({"proj":"utm", "zone":33, "datum":'WGS84'},{"proj":'latlong',"datum":'WGS84'})
    lonlat = xy_to_lonlat_transformer.transform(x,y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Austin, TX center longitude={}, latitude={}'.format(austin_center[1], austin_center[0]))
x, y = lonlat_to_xy(austin_center[1], austin_center[0])
print('Austin, TX center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Austin, TX center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Austin, TX center longitude=-97.740405, latitude=30.274687
Austin, TX center UTM X=-6445403.930149286, Y=13718424.016216282
Austin, TX center longitude=-97.74040500000329, latitude=30.27468699999805


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.

In [6]:
austin_center_x, austin_center_y = lonlat_to_xy(austin_center[1], austin_center[0]) # City center in Cartesian coordinates
print('Austin, TX center x={}, y={}'.format(austin_center_x, austin_center_y))

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = austin_center_x - 6000
x_step = 600
y_min = austin_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(austin_center_x, austin_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

Austin, TX center x=-6445403.930149286, y=13718424.016216282
364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [7]:
!pip install folium

import folium
print("Libraries Loaded")

Libraries Loaded


In [8]:
map_austin = folium.Map(location=austin_center, zoom_start=13)
folium.Marker(austin_center, popup='Texas State Capitol').add_to(map_austin)
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_austin) 
    folium.Circle([lat, lon], radius=300, color='orange', fill=False).add_to(map_austin)
map_austin

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from the Texas State Capitol.

Let's now use MapQuest API to get approximate addresses of those locations.

In [9]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'http://open.mapquestapi.com/geocoding/v1/reverse?key={}&location={},{}&includeRoadMetadata=true&includeNearestIntersection=true'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('MapQuest API JSON result =>', response)
        results = response['results']
        address = results[0]['locations'][0]['street']
        return address
    except:
        return None

addr = get_address(mapquest_api_key, austin_center[0], austin_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(austin_center[0], austin_center[1], addr))

Reverse geocoding check
-----------------------
Address of [30.274687, -97.740405] is: Texas State Capitol


In [10]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(mapquest_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [11]:
addresses[160:180]

['North Interstate Highway 35 Service Road',
 'Red River Street',
 'East 15th Street',
 'Colorado Street',
 '1809 San Antonio Street',
 'West 21st Street',
 '910 West 22nd Street',
 '2408 Leon Street',
 '2419 Wooldridge Drive',
 '1412 Wathen Avenue',
 '1602 Ethridge Avenue',
 '28 Margranita Crescent',
 'Lady Bird Lake Hike and Bike Trail',
 '2303 Canterbury Street',
 '94 Anthony Street',
 'East 2nd Street',
 '313 Comal Street',
 'East 5th Street',
 '1001 East 7th Street',
 '814 North Interstate Highway 35 Service Road']

In [12]:
df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

print(df_locations.shape)
df_locations.head(10)

(364, 6)


,Address,Latitude,Longitude,X,Y,Distance from center
0,3701 Airport Boulevard,30.286938,-97.705444,-6.447204e+06,1.371271e+07,5992.495307
1,3848 Airport Boulevard,30.289468,-97.707842,-6.446604e+06,1.371271e+07,5840.376700
2,3903 Brookview Road,30.291998,-97.710241,-6.446004e+06,1.371271e+07,5747.173218
3,4003 Maplewood Avenue,30.294529,-97.712639,-6.445404e+06,1.371271e+07,5715.767665
4,1301 Wilshire Boulevard,30.297059,-97.715038,-6.444804e+06,1.371271e+07,5747.173218
5,North Interstate Highway 35 Service Road,30.299590,-97.717438,-6.444204e+06,1.371271e+07,5840.376700
6,1003 East 43rd Street,30.302121,-97.719837,-6.443604e+06,1.371271e+07,5992.495307
7,1804 Mc Kinley Avenue,30.281341,-97.704372,-6.448104e+06,1.371323e+07,5855.766389
8,2105 Rountree Drive,30.283870,-97.706770,-6.447504e+06,1.371323e+07,5604.462508
9,2928 Manor Road,30.286400,-97.709168,-6.446904e+06,1.371323e+07,5408.326913
